In [ ]:
import cv2
import numpy as np
import time
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

def process_and_predict(frame, model, target_size=(224, 224)):
    """
    Processes a single frame and predicts using the model.

    Args:
        frame (np.array): Captured frame from the webcam.
        model: Loaded Keras model.
        target_size (tuple): Target size for resizing the frame.

    Returns:
        str: Feedback based on the prediction.
    """
    # Resize the frame to the model's input size
    resized_frame = cv2.resize(frame, target_size)
    normalized_frame = resized_frame / 255.0  # Normalize pixel values
    frame_array = img_to_array(normalized_frame)  # Convert to array
    frame_array = np.expand_dims(frame_array, axis=0)  # Add batch dimension

    # Predict using the model
    prediction = model.predict(frame_array)[0][0]

    # Provide feedback based on prediction
    if prediction > 0.5:
        return "Non-Defective", prediction
    else:
        return "Defective", prediction

def run_camera_with_model(model_path, interval=1, camera_index=0):
    """
    Runs the webcam, takes pictures at regular intervals, and processes them.

    Args:
        model_path (str): Path to the saved Keras model.
        interval (int): Time interval (in seconds) between captures.
        camera_index (int): Index of the webcam.
    """
    # Load the saved model
    print("Loading model...")
    model = load_model(model_path)
    print("Model loaded successfully.")

    # Open the webcam
    cap = cv2.VideoCapture(1)
    if not cap.isOpened():
        raise IOError("Cannot open the webcam")

    print("Webcam opened. Press 'ESC' to exit.")
    try:
        while True:
            # Capture frame
            ret, frame = cap.read()
            if not ret:
                print("Failed to capture frame. Exiting...")
                break

            # Process and predict
            feedback, confidence = process_and_predict(frame, model)
            print(f"Prediction: {feedback} (Confidence: {confidence:.2f})")

            # Display the frame with feedback
            cv2.putText(frame, f"Prediction: {feedback}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Confidence: {confidence:.2f}", (10, 60),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow("Webcam Feed", frame)

            # Wait for the interval or ESC key to exit
            if cv2.waitKey(interval * 1000) & 0xFF == 27:  # ESC key
                print("Exiting...")
                break

    finally:
        # Release resources
        cap.release()
        cv2.destroyAllWindows()

# Run the script
if __name__ == "__main__":
    model_path = "my_model_2.0.h5"  # Path to your saved model
    run_camera_with_model(model_path, interval=1)  # Captures every second
